# Using the Optimal Power Flow

This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

We don't want to deal with loading constraints in this first example, so we set them very high:

In [9]:
vm_max = 1.05
vm_min = 0.95
max_line = 1000
max_trafo = 1000

Let's create our basic example network:

In [24]:
import pandapower as pp
net = pp.create_empty_network()
pp.create_bus(net, max_vm_pu=vm_max, min_vm_pu=vm_min, vn_kv=10.)
pp.create_bus(net, max_vm_pu=vm_max, min_vm_pu=vm_min, vn_kv=.4)
pp.create_bus(net, max_vm_pu=vm_max, min_vm_pu=vm_min, vn_kv=.4)
pp.create_bus(net, max_vm_pu=vm_max, min_vm_pu=vm_min, vn_kv=.4)
pp.create_transformer_from_parameters(net, 0, 1, vsc_percent=3.75,tp_max=2, vn_lv_kv=0.4,shift_degree=150, 
tp_mid=0,vn_hv_kv=10.0, vscr_percent=2.8125,tp_pos=0, tp_side='hv', tp_min=-2,tp_st_percent=2.5,
i0_percent=0.68751,sn_kva=16.0, pfe_kw=0.11, name=None,in_service=True, index=None, max_loading_percent=200)
pp.create_ext_grid(net, 0)
pp.create_line_from_parameters(net, 1, 2, 1, name='line1', r_ohm_per_km=0.876,c_nf_per_km=260.0, imax_ka=0.123, x_ohm_per_km=0.1159876)
pp.create_line_from_parameters(net, 2, 3, 1, name='line2', r_ohm_per_km=0.876,c_nf_per_km=260.0, imax_ka=0.123, x_ohm_per_km=0.1159876,max_loading_percent=100000)

1

We add a generator, which should be curtailed as less as possible. To model its cost function, we add a negative cost_per_kwh

In [25]:
pp.create_gen(net, 3, p_kw=-10, controllable=True, max_p_kw=-25, min_p_kw=-5, max_q_kvar=50, min_q_kvar=-50, cost_per_kw = -100)

0

Now we can run our first optimal power flow:

In [26]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [27]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-17.665028,28.611099,23.359787,1.05


In [28]:
net.res_ext_grid

,p_kw,q_kvar
0,4.539148,-31.657901


In [29]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,4.539148,-31.657901
1,0.961046,3.744985,0.000000,0.000000
2,0.990862,13.988421,0.000000,0.000000
3,1.050000,23.359787,-17.665028,28.611099


As you can see, the generator is set to it's maximum value where the voltage constraint can still be met.

Another application of the OPF is to minimze the generator output while minimizing the power supply from the external grid. 
 Let's create a load first, so we have something we need to supply

In [16]:
pp.create_load(net, 2, p_kw = 20)

0

Modify the cost function by setting new cost_per_kw values. We assume, that it is more important to minimize the power supply from the external grid, thats why we simply double the costs.

In [17]:
net.gen.cost_per_kw=100
net.ext_grid.cost_per_kw=200

Run optimal power flow:

In [18]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Check the results:

In [19]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-18.006614,4.661794,3.784141,1.05


In [20]:
net.res_ext_grid

,p_kw,q_kvar
0,-4.11438,-4.957365


In [21]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-4.114380,-4.957365
1,0.985182,0.140744,0.000000,0.000000
2,0.960023,1.593161,20.000000,0.000000
3,1.050000,3.784141,-18.006614,4.661794


So we see, that the most part of the load is supplied by the generator, but in order to keep the voltage limits, the generator can not feed in 100%.

Let's now deal with loading constraints, too. Let's first check the loading of the previous calculation:

In [22]:
net.res_trafo.loading_percent

0    40.264552
Name: loading_percent, dtype: float64

In [23]:
net.res_line.loading_percent

0     7.490070
1    20.787613
Name: loading_percent, dtype: float64

So the line between the load and the generator has a loading of 20%, which should be okay. But let's still use this one as an example on how to reduce line loading. Let's set the maximum loading to 19% and see, what happens:

In [16]:
net.line.max_loading_percent.loc[1]=19

pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Check the results:

In [17]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-16.94977,1.31579,1.129683,1.05


In [18]:
net.res_ext_grid

,p_kw,q_kvar
0,-4.769387,-1.53628


In [19]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-4.769387,-1.53628
1,0.989342,-0.266932,0.000000,0.00000
2,0.962701,0.027057,20.000000,0.00000
3,1.050000,1.129683,-16.949770,1.31579


In [20]:
net.res_trafo.loading_percent

0    31.316935
Name: loading_percent, dtype: float64

In [21]:
net.res_line.loading_percent

0     5.762240
1    18.999994
Name: loading_percent, dtype: float64

We see, that the loading constraint was met by further reducing the generator output.

If we want to add a generator, that won't be considered as a flexibility, we can tighten it's power boundaries like this:

In [22]:
pp.create_bus(net, max_vm_pu=vm_max, min_vm_pu=vm_min, vn_kv=.4)
pp.create_line_from_parameters(net, 3, 4, 1, name='line3', r_ohm_per_km=0.876, c_nf_per_km=260.0, imax_ka=0.123, x_ohm_per_km=0.1159876, max_loading_percent=100000)
pp.create_gen(net, 4, p_kw=-2, controllable=True, max_p_kw=-2, min_p_kw=-2, max_q_kvar=500, min_q_kvar=-500)
pp.runopp(net, verbose=False)

Check the results:

In [23]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-14.85765,-2.607067,1.983693,1.043463
1,-2.00000,5.101424,3.518236,1.050000


In [24]:
net.res_ext_grid

,p_kw,q_kvar
0,-5.067983,-2.738742


In [25]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-5.067983,-2.738742
1,0.986947,-0.171745,0.000000,0.000000
2,0.957842,0.506825,20.000000,0.000000
3,1.043463,1.983693,-14.857650,-2.607067
4,1.050000,3.518236,-2.000000,5.101424


In [26]:
net.res_trafo.loading_percent

0    36.0041
Name: loading_percent, dtype: float64

In [27]:
net.res_line.loading_percent

0     6.654287
1    18.999935
2     6.123830
Name: loading_percent, dtype: float64

The OPF resepcted the boundaries of the second generator and set it to the exact value. However, in some cases, this may lead to convergence problems and the boundaries should be opened up a little bit.